In [2]:
import numpy as np
from pymatgen.core import Structure, Lattice, PeriodicSite, DummySpecie
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import MinimumDistanceNN, CrystalNN 

In [ ]:
'''# Load defective structure
defective_struct = Structure.from_file("low_density_defects/MoS2/MoS2cifs/6141cf1a1d648932fbc34297.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("low_density_defects/MoS2/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])'''


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 48 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [ ]:
# Load defective structure
defective_struct = Structure.from_file("Datasets/high_density_defects/MoS2_500/highMoS2cifs/MoS2_Mo56W4Se3S117_b593cd47-47e7-4a55-9a27-466f5c3f661c.cif")
struct_lattice = defective_struct.lattice

# Get reference structure
ref_unit_cell = Structure.from_file("Datasets/high_density_defects/MoS2_500/MoS2.cif")
reference_struct = ref_unit_cell.make_supercell([8,8,1])


/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 45 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [29]:
def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict

In [ ]:
def get_defect_structure(defective_dict, reference_dict, struct_lattice): 
    defect_site = []

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie:  # Substitution case
                    defect_site.append(PeriodicSite(
                        species= def_site.species,
                        coords= def_site.frac_coords,
                        coords_are_cartesian=False,
                        lattice= struct_lattice,
                        properties= {"original_new_am": (ref_site.specie.Z, def_site.specie.Z)}
                    ))

        if not matching:           # Vacancy case
            defect_site.append(PeriodicSite(
                species=DummySpecie(),
                coords = ref_coords,
                coords_are_cartesian=False,
                lattice = struct_lattice,
                properties = {"original_new_am": (ref_site.specie.Z, 0)}
                ))

    # Create a structure with the defect sites
    defect_struct = Structure.from_sites(defect_site)
    return defect_struct

# Convert structures to dictionaries
defective_dict = struct_to_dict(defective_struct)
reference_dict = struct_to_dict(reference_struct)

# Get defect structure
defect_struct = get_defect_structure(defective_dict, reference_dict, struct_lattice)


In [38]:
def get_defects(defective_struct, reference_struct):
    # struct to dict
    defective_dict = struct_to_dict(defective_struct)
    reference_dict = struct_to_dict(reference_struct)

    # Get lattice of defective structure
    structure_lattice = defective_struct.lattice

    # Define list to add all defect sites
    defect_sites_to_struct = []
    for_index_comparison = []

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie:  # Substitution case
                    for_index_comparison.append(def_site)

                    sub_site = PeriodicSite(
                        species=def_site.specie,
                        coords= def_site.frac_coords,
                        coords_are_cartesian= False,
                        lattice= structure_lattice,
                        properties= {"orig_new_am":(ref_site.specie.Z, def_site.specie.Z)}
                    )
                    defect_sites_to_struct.append(sub_site)

        if not matching:           # Vacancy case
            # Add site to defective structure
            vacant_specie = DummySpecie()
            the_coords = ref_coords
            defective_struct.append(vacant_specie, the_coords)

            for_index_comparison.append(PeriodicSite(
                species= vacant_specie,
                coords=the_coords,
                coords_are_cartesian=False, 
                lattice=structure_lattice
                ))

            # Create a proper vacant site
            vacant_site = PeriodicSite(
                species= vacant_specie,
                coords=the_coords,
                coords_are_cartesian=False, 
                lattice=structure_lattice,
                properties={"orig_new_am": (ref_site.specie.Z, 0)}
                )
            defect_sites_to_struct.append(vacant_site)
            

    # Till this point every defect site is in the defective structure
    return defective_struct, defect_sites_to_struct, for_index_comparison


# Get defect structure
new_defective_struct, the_defect_sites,to_compare = get_defects(defective_struct, reference_struct)


In [39]:
print(new_defective_struct)

Full Formula (X12 Mo56 W4 Se3 S117)
Reduced Formula: X12Mo56W4(SeS39)3
abc   :  25.522526  25.522526  20.000000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (192)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Mo    0.041667  0.083333  0.185988
  1  Mo    0.041667  0.208333  0.185988
  2  Mo    0.041667  0.333333  0.185988
  3  Mo    0.041667  0.458333  0.185988
  4  Mo    0.041667  0.583333  0.185988
  5  Mo    0.041667  0.708333  0.185988
  6  Mo    0.041667  0.833333  0.185988
  7  Mo    0.166667  0.083333  0.185988
  8  Mo    0.166667  0.333333  0.185988
  9  Mo    0.166667  0.583333  0.185988
 10  Mo    0.166667  0.708333  0.185988
 11  Mo    0.166667  0.833333  0.185988
 12  Mo    0.166667  0.958333  0.185988
 13  Mo    0.291667  0.083333  0.185988
 14  Mo    0.291667  0.208333  0.185988
 15  Mo    0.291667  0.333333  0.185988
 16  Mo    0.291667  0.458333  0.185988
 17  Mo    0.291667  0.583333  0.1

Let's get the nearest neigbors of the defects in the defective structure

In [35]:
# Define an instance responsible for getting the nearest neighbors
cnn = CrystalNN()

list_of_sites = new_defective_struct.sites
print(list_of_sites)


[PeriodicSite: Mo0 (Mo) (1.276e-05, 1.842, 3.72) [0.04167, 0.08333, 0.186], PeriodicSite: Mo1 (Mo) (-1.595, 4.605, 3.72) [0.04167, 0.2083, 0.186], PeriodicSite: Mo2 (Mo) (-3.19, 7.368, 3.72) [0.04167, 0.3333, 0.186], PeriodicSite: Mo3 (Mo) (-4.785, 10.13, 3.72) [0.04167, 0.4583, 0.186], PeriodicSite: Mo4 (Mo) (-6.381, 12.89, 3.72) [0.04167, 0.5833, 0.186], PeriodicSite: Mo5 (Mo) (-7.976, 15.66, 3.72) [0.04167, 0.7083, 0.186], PeriodicSite: Mo6 (Mo) (-9.571, 18.42, 3.72) [0.04167, 0.8333, 0.186], PeriodicSite: Mo7 (Mo) (3.19, 1.842, 3.72) [0.1667, 0.08333, 0.186], PeriodicSite: Mo8 (Mo) (8.508e-06, 7.368, 3.72) [0.1667, 0.3333, 0.186], PeriodicSite: Mo9 (Mo) (-3.19, 12.89, 3.72) [0.1667, 0.5833, 0.186], PeriodicSite: Mo10 (Mo) (-4.785, 15.66, 3.72) [0.1667, 0.7083, 0.186], PeriodicSite: Mo11 (Mo) (-6.381, 18.42, 3.72) [0.1667, 0.8333, 0.186], PeriodicSite: Mo12 (Mo) (-7.976, 21.18, 3.72) [0.1667, 0.9583, 0.186], PeriodicSite: Mo13 (Mo) (6.381, 1.842, 3.72) [0.2917, 0.08333, 0.186], Peri

In [40]:
list_of_neighbors = []

# Get the nearest neighbors of each defect site

for defect_site in to_compare:
    the_index = list_of_sites.index(defect_site)
    neighbors = cnn.get_nn(new_defective_struct, the_index)
    list_of_neighbors.append(neighbors)

print(list_of_neighbors)


AttributeError: 